In [11]:
import serial
import datetime

In [2]:
! ls /dev/tty.*

/dev/tty.Bluetooth-Incoming-Port /dev/tty.usbmodem14212


# First try

See my repository, `micropython_pyboard`.

When I make a change in the PyBoard code I have to eject it from Finder, unplug the USB, plug USB back in, and then the new code is executing on the PyBoard.

## PyBoard code

    import micropython
    micropython.alloc_emergency_exception_buf(100)

    import pyb
    import micropython

    class Heartbeat(object):

        def __init__(self):
            self.tick = 0
            self.led = pyb.LED(4) # 4 = Blue
            tim = pyb.Timer(4)
            tim.init(freq=10)
            tim.callback(self.heartbeat_cb)

        def heartbeat_cb(self, tim):
            if self.tick <= 3:
                self.led.toggle()
            self.tick = (self.tick + 1) % 10

    class serial_speed_test(object):

        def __init__(self, freq_Hz):
            self.tick = 0
            self.tick_ready = False
            self.micros_timer = micros_timer.counter()
            self.freq_Hz = freq_Hz
            tim1 = pyb.Timer(1)
            tim1.init(freq=freq_Hz)
            tim1.callback(self.serial_speed_test_cb)

        def serial_speed_test_cb(self, tim1):
            self.micros_timer = micros_timer.counter()
            self.tick_ready = True
            #print(micros_timer.counter(), ',', 40*self.tick)
            self.tick = (self.tick + 1) % 100

    micropython.alloc_emergency_exception_buf(100)

    micros_timer = pyb.Timer(2, prescaler=83, period=0x3ffffff)

    usb = pyb.USB_VCP()
    Heartbeat()
    sst = serial_speed_test(2)
    write_flag = False

    while True:
        if usb.any():
            input = usb.readline()
            usb.write(input)
            if input.startswith('start'):
                write_flag = True
            elif input.startswith('stop'):
                write_flag = False
        if write_flag:
            if sst.tick_ready:
                s = "%d,%d\n" % (sst.tick, sst.micros_timer)
                usb.write(s)
                sst.tick_ready = False


## Communicate over USB serial

In [19]:
with serial.Serial('/dev/tty.usbmodem14212', 115200) as ser:
    ser.write('start\n'.encode())
    start_time = time.time()
    for i in range(25):
        time.sleep(0.05)
        print(time.time() - start_time, ser.readline().decode().strip())
    ser.write('stop\n'.encode())
    print(time.time() - start_time, ser.readline().decode().strip())

0.051561832427978516 start
0.10205578804016113 62,42838144
0.15441679954528809 63,43038144
0.24419188499450684 64,43238144
0.4416360855102539 65,43438144
0.6398649215698242 66,43638144
0.8402459621429443 67,43838144
1.0417258739471436 68,44038144
1.2433960437774658 69,44238144
1.4411258697509766 70,44438144
1.641869068145752 71,44638144
1.8431668281555176 72,44838144
2.042341709136963 73,45038144
2.2398269176483154 74,45238144
2.4444479942321777 75,45438144
2.643293857574463 76,45638144
2.840578079223633 77,45838144
3.0442097187042236 78,46038144
3.2444889545440674 79,46238144
3.444139003753662 80,46438144
3.641921043395996 81,46638144
3.8400399684906006 82,46838144
4.04105806350708 83,47038144
4.240850925445557 84,47238144
4.443668842315674 85,47438144
4.5895538330078125 stop
